# Multi-Modal Fine-Tuning


## Install and import libraries

In [1]:
!pip install matplotlib -q -U
!pip install datasets -q -U
!pip install -q bitsandbytes sentencepiece  accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install hf_transfer -q -U
!pip install pickleshare -q

In [ ]:
# Allows for faster downloads
%env HF_HUB_ENABLE_HF_TRANSFER=1

In [3]:
import os

if not os.path.isdir('LLaVA'):
    !git clone https://github.com/haotian-liu/LLaVA.git
else:
    print("LLaVA already exists. Skipping clone.")

Klone nach 'LLaVA'...
remote: Enumerating objects: 2297, done.
remote: Total 2297 (delta 0), reused 0 (delta 0), pack-reused 2297 (from 1)
Empfange Objekte: 100% (2297/2297), 13.71 MiB | 12.02 MiB/s, fertig.
Löse Unterschiede auf: 100% (1405/1405), fertig.


In [9]:
import re

# Define the path to the builder.py file
file_path = '/Users/fabian.fuerst/Documents/GitHub/Fine-tuned-LLaVA-Vision-and-Language/fine-tuning/LLaVa_1.6_7B/LLaVA/llava/model/builder.py'

# Read the content of the file
with open(file_path, 'r') as file:
    content = file.read()

# Regular expression to find the block between 'vision_tower = model.get_vision_tower()' and 'vision_tower.image_processor'
pattern_block = (
    r'(vision_tower = model.get_vision_tower\(\)\n)'
    r'.*?' # Non-greedy match for any characters
    r'(image_processor = vision_tower.image_processor)'
)

replacement_block = (
    r'\1' # Keep the first line unchanged
    '       if not vision_tower.is_loaded:\n'
    '           print(\'vision_tower is not loades so loading now\')\n'
    '           vision_tower.load_model(device_map=device_map)\n'
    '           vision_tower.to(device=device_map, dtype=torch.bfloat16)\n'
    '       else:\n'
    '           print(\'vision_tower is already loaded\')\n'
    r'      \2' # Keep the last line unchanged
)

# Replace the specific block
content = re.sub(pattern_block, replacement_block, content, flags=re.DOTALL)

# Write the content back to the file
with open(file_path, 'w') as file:
    file.write(content)

print('File modified successfully.')

File modified successfully.


In [10]:
import re

# Define the path to the builder.py file
file_path = '/Users/fabian.fuerst/Documents/GitHub/Fine-tuned-LLaVA-Vision-and-Language/fine-tuning/LLaVa_1.6_7B/LLaVA/llava/model/builder.py'

# Read the content of the file
with open(file_path, 'r') as file:
    content = file.read()

# Regular expression to find 'float16' and replace it with 'bfloat16'
pattern = r'(?<!b)float16'

# CHeck if there are any matches
if re.search(pattern, content):
    # Replace all matches
    modified_content = re.sub(pattern, 'bfloat16', content)

    # Write the content back to the file
    with open(file_path, 'w') as file:
        file.write(content)

    print('File modified successfully.')
else:
    print('No modification needed.')

No modification needed.


In [6]:
%cd LLaVA

/Users/fabian.fuerst/Documents/GitHub/Fine-tuned-LLaVA-Vision-and-Language/fine-tuning/LLaVa_1.6_7B/LLaVA


In [7]:
# Takes quite a while to run
!pip install -e . -q

In [11]:
!pip install protobuf -q -U
!pip installl --upgrade Pillow -q
!pip install -e '.[train]' -q
!pip install flash-attn --no-build-isolation -q

ERROR: unknown command "installl" - maybe you meant "install"
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/jw/9kbqqtt11wj27s6z8wmgk_yw0000gp/T/pip-install-2jbcb61h/deepspeed_b0e5288316eb4ed2be57c7e5651e6737/setup.py", line 38, in <module>
          from op_builder.all_ops import ALL_OPS
        File "/private/var/folders/jw/9kbqqtt11wj27s6z8wmgk_yw0000gp/T/pip-install-2jbcb61h/deepspeed_b0e5288316eb4ed2be57c7e5651e6737/op_builder/all_ops.py", line 29, in <module>
          builder = get_accelerator().create_op_builder(member_name)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/private/var/folders/jw/9kbqqtt11wj27s6z8wmgk_yw0000gp/T/pip-install-2jbcb61h/deepspeed_b0e5288316eb4ed

## Load the Model

In [1]:
import torch
from datasets import load_dataset
import transformers
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
import torchvision.transforms as transforms

/Users/fabian.fuerst/Documents/GitHub/Fine-tuned-LLaVA-Vision-and-Language/.conda/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [ ]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_path = 'liuhaotian/llava-v1.6-mistral-7b'

model_name = get_model_name_from_path(model_path)

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=model_name,
    cache_dir='',
    use_flash_attn=True,
)